In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model.parameters(),
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    model=model,
    use_diagonal=False,
    c1=1e-4,
    tau=0.5,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.6343417763710022
epoch:  1, loss: 0.11095446348190308
epoch:  2, loss: 0.04510517045855522
epoch:  3, loss: 0.022239182144403458
epoch:  4, loss: 0.0157375056296587
epoch:  5, loss: 0.014483723789453506
epoch:  6, loss: 0.009270602837204933
epoch:  7, loss: 0.008906678296625614
epoch:  8, loss: 0.007963468320667744
epoch:  9, loss: 0.007742570713162422
epoch:  10, loss: 0.00688402121886611
epoch:  11, loss: 0.00653162831440568
epoch:  12, loss: 0.0057039218954741955
epoch:  13, loss: 0.005500299856066704
epoch:  14, loss: 0.0046315062791109085
epoch:  15, loss: 0.004435374401509762
epoch:  16, loss: 0.0037464911583811045
epoch:  17, loss: 0.003506216686218977
epoch:  18, loss: 0.0033750480506569147
epoch:  19, loss: 0.0030802267137914896
epoch:  20, loss: 0.0029791456181555986
epoch:  21, loss: 0.002701696939766407
epoch:  22, loss: 0.002609194489195943
epoch:  23, loss: 0.002557491883635521
epoch:  24, loss: 0.0024495255202054977
epoch:  25, loss: 0.0023913779295980

In [6]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9813711866670917
Test metrics:  R2 = 0.9699278589986875


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.AGD(
    model.parameters(),
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    model=model,
    use_diagonal=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch] += loss

    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch - 1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience

    print(", loss: {}".format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.39195582270622253
epoch:  1, loss: 0.278076708316803
epoch:  2, loss: 0.23507434129714966
epoch:  3, loss: 0.19397634267807007
epoch:  4, loss: 0.05593416839838028
epoch:  5, loss: 0.014008278027176857
epoch:  6, loss: 0.011346837505698204
epoch:  7, loss: 0.010078780353069305
epoch:  8, loss: 0.009671495296061039
epoch:  9, loss: 0.008871881291270256
epoch:  10, loss: 0.00828420277684927
epoch:  11, loss: 0.008148652501404285
epoch:  12, loss: 0.007974503561854362
epoch:  13, loss: 0.007588700857013464
epoch:  14, loss: 0.007204002235084772
epoch:  15, loss: 0.006947745569050312
epoch:  16, loss: 0.006455664522945881
epoch:  17, loss: 0.0062392838299274445
epoch:  18, loss: 0.006113196723163128
epoch:  19, loss: 0.006031856406480074
epoch:  20, loss: 0.005978474393486977
epoch:  21, loss: 0.005813491530716419
epoch:  22, loss: 0.0057672662660479546
epoch:  23, loss: 0.005725297145545483
epoch:  24, loss: 0.005696759093552828
epoch:  25, loss: 0.005678044632077217
ep

In [8]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.9327147819997506
Test metrics:  R2 = 0.916334225814023
